# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Li  ->  S. Li  |  ['S. Li']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
F. Walter  ->  F. Walter  |  ['F. Walter']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
Arxiv has 53 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2403.17048


extracting tarball to tmp_2403.17048... done.
Retrieving document from  https://arxiv.org/e-print/2403.17903


extracting tarball to tmp_2403.17903... done.


Found 84 bibliographic references in tmp_2403.17903/main.bbl.
syntax error in line 396: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.17903-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.17903) | **Quasar Island - Three new $z\sim6$ quasars, including a lensed  candidate, identified with contrastive learning**  |
|| X. Byrne, et al. -- incl., <mark>E. Bañados</mark>, <mark>F. Walter</mark>, <mark>S. Belladitta</mark> |
|*Appeared on*| *2024-03-27*|
|*Comments*| *11 pages, 5 figures, 4 tables. Accepted 26 March 2024 for publication in MNRAS*|
|**Abstract**| Of the hundreds of $z\gtrsim6$ quasars discovered to date, only one is known to be gravitationally lensed, despite the high lensing optical depth expected at $z\gtrsim6$. High-redshift quasars are typically identified in large-scale surveys by applying strict photometric selection criteria, in particular by imposing non-detections in bands blueward of the Lyman-$\alpha$ line. Such procedures by design prohibit the discovery of lensed quasars, as the lensing foreground galaxy would contaminate the photometry of the quasar. We present a novel quasar selection methodology, applying contrastive learning (an unsupervised machine learning technique) to Dark Energy Survey imaging data. We describe the use of this technique to train a neural network which isolates an 'island' of 11 sources, of which 7 are known $z\sim6$ quasars. Of the remaining four, three are newly discovered quasars (J0109-5424, $z=6.07$; J0122-4609, $z=5.99$; J0603-3923, $z=5.94$), as confirmed by follow-up Gemini-South/GMOS and archival NTT/EFOSC2 spectroscopy, implying a 91 per cent efficiency for our novel selection method; the final object on the island is a brown dwarf. In one case (J0109-5424), emission below the Lyman limit unambiguously indicates the presence of a foreground source, though high-resolution optical/near-infrared imaging is still needed to confirm the quasar's lensed (multiply-imaged) nature. Detection in the g band has led this quasar to escape selection by traditional colour cuts. Our findings demonstrate that machine learning techniques can thus play a key role in unveiling populations of quasars missed by traditional methods. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.17048-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.17048) | **The Tip of the Red Giant Branch Distance Ladder and the Hubble Constant**  |
|| <mark>S. Li</mark>, R. L. Beaton |
|*Appeared on*| *2024-03-27*|
|*Comments*| *20 pages, 3 figures, Invited chapter for the edited book "Hubble Constant Tension" (Eds. E. Di Valentino and D. Brout, Springer Singapore, expected in 2024)*|
|**Abstract**| While the tip of the red giant branch (TRGB) has been used as a distance indicator since the early 1990's, its application to measure the Hubble Constant as a primary distance indicator occurred only recently. The TRGB is also currently at an interesting crossroads as results from the James Webb Space Telescope (JWST) are beginning to emerge. In this chapter, we provide a review of the TRGB as it is used to measure the Hubble constant. First, we provide an essential review of the physical and observational basis of the TRGB as well as providing a summary for its use for measuring the Hubble Constant. More attention is then given is then given to recent, but still pre-JWST, developments, including new calibrations and developments with algorithms. We also address challenges that arise while measuring a TRGB-based Hubble Constant. We close by looking forward to the exciting prospects from telescopes such as JWST and Gaia. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2403.17903.md
    + _build/html/tmp_2403.17903/./quasar_island_objects_this_work.png
    + _build/html/tmp_2403.17903/./seds.png
    + _build/html/tmp_2403.17903/./selection_criteria.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

#     _Quasar Island_ -- Three new $z\sim6$ quasars, including a lensed candidate, identified with contrastive learning

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2403.17903-b31b1b.svg)](https://arxiv.org/abs/2403.17903)<mark>Appeared on: 2024-03-27</mark> -  _11 pages, 5 figures, 4 tables. Accepted 26 March 2024 for publication in MNRAS_

</div>
<div id="authors">

X. Byrne, et al. -- incl., <mark>E. Bañados</mark>, <mark>F. Walter</mark>, <mark>S. Belladitta</mark>

</div>
<div id="abstract">

**Abstract:** Of the hundreds of $z\gtrsim6$ quasars discovered to date, only one is known to be gravitationally lensed, despite the high lensing optical depth expected at $z\gtrsim 6$ .High-redshift quasars are typically identified in large-scale surveys by applying strict photometric selection criteria, in particular by imposing non-detections in bands blueward of the $\text{Lyman-}\alpha$ line.Such procedures by design prohibit the discovery of lensed quasars, as the lensing foreground galaxy would contaminate the photometry of the quasar.We present a novel quasar selection methodology, applying contrastive learning (an unsupervised machine learning technique) to Dark Energy Survey imaging data.We describe the use of this technique to train a neural network which isolates an `island' of 11 sources, of which 7 are known $z\sim6$ quasars.Of the remaining four, three are newly discovered quasars (J0109--5424, $z=6.07$ ; J0122--4609, $z=5.99$ ; J0603--3923, $z=5.94$ ), as confirmed by follow-up Gemini-South/GMOS and archival NTT/EFOSC2 spectroscopy, implying a $91$ per cent efficiency for our novel selection method; the final object on the island is a brown dwarf.In one case (J0109--5424), emission below the Lyman limit unambiguously indicates the presence of a foreground source, though high-resolution optical/near-infrared imaging is still needed to confirm the quasar's lensed (multiply-imaged) nature.Detection in the _g_ band has led this quasar to escape selection by traditional colour cuts.Our findings demonstrate that machine learning techniques can thus play a key role in unveiling populations of quasars missed by traditional methods.

</div>

<div id="div_fig1">

<img src="tmp_2403.17903/./quasar_island_objects_this_work.png" alt="Fig4" width="100%"/>

**Figure 4. -** 
    Confirmation spectra of the four unpublished objects of `Quasar Island' studied in this work with EFOSC2/NTT and Gemini-South/GMOS.
    The spectra are all flux-calibrated and scaled to the respective DES _z_-band magnitudes (see Table \ref{tab:quasarislandmags}).
    We show the extracted 1D error array in red.
    We confirm the discovery of three new quasars at $z\sim 6$, bringing the completeness of `Quasar Island' to $91$ per cent.
    The remaining object is a cool dwarf whose spectrum closely matches that of an M9 dwarf \citep{Burgasser2008}.
    For J0109--5424, we highlight in blue the presence of flux in the Lyman-$\alpha$ forest and below the Lyman limit (orange dashed line), which we interpret as evidence for a foreground lensing galaxy (see Section \ref{sec:lensed_quasar}); we also identify possible $\ion${C}{IV} and $\ion${Si}{IV} absorptions.
    For the other two quasars, no flux is detected below the Lyman limit.  (*fig:all_spectra*)

</div>
<div id="div_fig2">

<img src="tmp_2403.17903/./seds.png" alt="Fig5" width="100%"/>

**Figure 5. -** 
DES+VHS+WISE imaging and best-fitting spectra of the candidates' photometry for galaxy, quasar and lensed quasar templates based on BAGPIPES galaxy models \citep{carnall18} and QSOGEN quasar models \citep{Temple2021,temple22} as described in Section \ref{sec:lensed_quasar}.
The images are each $7.5 \text{arcsec}$ across.
Integrated fluxes for _grizY_, _J_, _K_, _W1_ and _W2_ bands are indicated in the coloured circles.
The observed fluxes measured by the three surveys are indicated by the black squares with error bars.
The BICs for the model fits are given in the legend.
For J0109--5424, a galaxy+quasar template provides a significantly better fit than a pure galaxy or quasar template, and the source is discernable in the _g_ band imaging (top row, far left).
For J0603--3923 and J0122--4609, the best-fitting galaxy+quasar models are almost identical to the best-fitting quasar models, and there is no clear source in the _g_ band.
 (*fig:fits*)

</div>
<div id="div_fig3">

<img src="tmp_2403.17903/./selection_criteria.png" alt="Fig1" width="100%"/>

**Figure 1. -** Photometric selection criteria.
The black points are sources selected from three random DES tiles, most of which are likely stars.
The blue stars indicate brown dwarfs from \citet{kirkpatrick11} and \citet{best15} cross-matched to DES.
Several known high-redshift quasars are plotted, as well as a model quasar track generated using QSOGEN\citep{temple22}; both are coloured by redshift.
Our liberal photometric selection criteria are shown by the dashed lines: $\mathit{i}-\mathit{z}>1$ and $\mathit{W1}-\mathit{W2} > -0.14$(both AB).
 (*fig:colours*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2403.17903"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

393  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
